In [1]:
# Start with loading all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random

from wordcloud import WordCloud, STOPWORDS


In [2]:
#creating the table
df = pd.read_csv(r'C:\Users\flowe\Downloads\Data Model - Pizza Sales.csv')


###### <img src="https://giffiles.alphacoders.com/148/148200.gif" width = "800" align="center"/>

# BB's Buongustaio Pizza Restaurante 2015 Annual Review

In this project we'll be analysing store data from popular mom and pop restaurant BB's Buongustaio Pizza Restaurante. We'll be collaborating with stakeholders to create an interactive dashboard of their 2015 sales and noting suggestions for BB's marketing department and store managers to create a sales strategy.



In [3]:
df.head()

,order_details_id,table_number,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,1/1/2015,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,1/1/2015,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,1/1/2015,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,1/1/2015,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,1/1/2015,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


Taking a general look at the data, some of the information collected from the store is the customer order time, the date the order was placed, the name, type and size of the pizza, as well as the total price of each order recieved.


In [4]:
df.columns

Index(['order_details_id', 'table_number', 'pizza_id', 'quantity',
       'order_date', 'order_time', 'unit_price', 'total_price', 'pizza_size',
       'pizza_category', 'pizza_ingredients', 'pizza_name'],
      dtype='object')

In [5]:
df.dtypes

order_details_id       int64
table_number           int64
pizza_id              object
quantity               int64
order_date            object
order_time            object
unit_price           float64
total_price          float64
pizza_size            object
pizza_category        object
pizza_ingredients     object
pizza_name            object
dtype: object

### Defining columns:
- table_number: Unique identifier for each order placed by a table
- order_details_id: Unique identifier for each pizza placed within each order (pizzas of the same type and size are kept in the same row, and the quantity increases)
- pizza_id: Unique key identifier that ties the pizza ordered to its details, like size and price
- quantity: Quantity ordered for each pizza of the same type and size
- order_date: Date the order was placed (entered into the system prior to cooking & serving)
- order_time: Time the order was placed (entered into the system prior to cooking & serving)
- unit_price: Price of the pizza in USD
- total_price: unit_price * quantity
- pizza_size: Size of the pizza (Small, Medium, Large, X Large, or XX Large)
- pizza_type: Unique key identifier that ties the pizza ordered to its details, like size and price
- pizza_ingredients: ingredients used in the pizza as shown in the menu (they all include Mozzarella Cheese, even if not specified; and they all include Tomato Sauce, unless another sauce is specified)
- pizza_name: Name of the pizza as shown in the menu

As we move forward to answer certain questions, we will have to construct several features suitable for that particular query. 

For now, we will construct two features: 
 - df['day of the week']: The day of the week the order was placed
 - df['hour_of_the_week']: The time of day the order was placed by the whole hour 
 - df['month']: The month the order was placed
 
 These features will help us determine the top selling days of the week, months and times of day. 


In [38]:
# creating day of the week column
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
df["day_of_week"] = df["order_date"].dt.dayofweek
df


,order_details_id,table_number,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name,day_of_week,hour_of_day,month_of_sale
0,1,1,hawaiian_m,1,2015-01-01,2023-03-24 11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza,3,11,1
1,2,2,classic_dlx_m,1,2015-01-01,2023-03-24 11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza,3,11,1
2,3,2,five_cheese_l,1,2015-01-01,2023-03-24 11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza,3,11,1
3,4,2,ital_supr_l,1,2015-01-01,2023-03-24 11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza,3,11,1
4,5,2,mexicana_m,1,2015-01-01,2023-03-24 11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza,3,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48615,48616,21348,ckn_alfredo_m,1,2015-12-31,2023-03-24 21:23:10,16.75,16.75,M,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A...",The Chicken Alfredo Pizza,3,21,12
48616,48617,21348,four_cheese_l,1,2015-12-31,2023-03-24 21:23:10,17.95,17.95,L,Veggie,"Ricotta Cheese, Gorgonzola Piccante Cheese, Mo...",The Four Cheese Pizza,3,21,12
48617,48618,21348,napolitana_s,1,2015-12-31,2023-03-24 21:23:10,12.00,12.00,S,Classic,"Tomatoes, Anchovies, Green Olives, Red Onions,...",The Napolitana Pizza,3,21,12
48618,48619,21349,mexicana_l,1,2015-12-31,2023-03-24 22:09:54,20.25,20.25,L,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza,3,22,12


In [7]:
#creating hour column
df['order_time'] = pd.to_datetime(df['order_time'], errors='coerce')
df["hour_of_day"] = df["order_time"].dt.hour
df


,order_details_id,table_number,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name,day_of_week,hour_of_day
0,1,1,hawaiian_m,1,2015-01-01,2023-03-24 11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza,3,11
1,2,2,classic_dlx_m,1,2015-01-01,2023-03-24 11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza,3,11
2,3,2,five_cheese_l,1,2015-01-01,2023-03-24 11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza,3,11
3,4,2,ital_supr_l,1,2015-01-01,2023-03-24 11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza,3,11
4,5,2,mexicana_m,1,2015-01-01,2023-03-24 11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza,3,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48615,48616,21348,ckn_alfredo_m,1,2015-12-31,2023-03-24 21:23:10,16.75,16.75,M,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A...",The Chicken Alfredo Pizza,3,21
48616,48617,21348,four_cheese_l,1,2015-12-31,2023-03-24 21:23:10,17.95,17.95,L,Veggie,"Ricotta Cheese, Gorgonzola Piccante Cheese, Mo...",The Four Cheese Pizza,3,21
48617,48618,21348,napolitana_s,1,2015-12-31,2023-03-24 21:23:10,12.00,12.00,S,Classic,"Tomatoes, Anchovies, Green Olives, Red Onions,...",The Napolitana Pizza,3,21
48618,48619,21349,mexicana_l,1,2015-12-31,2023-03-24 22:09:54,20.25,20.25,L,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza,3,22


In [8]:
#creating month column
df['order_time'] = pd.to_datetime(df['order_time'], errors='coerce')
df["month_of_sale"] = df["order_date"].dt.month
df

,order_details_id,table_number,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name,day_of_week,hour_of_day,month_of_sale
0,1,1,hawaiian_m,1,2015-01-01,2023-03-24 11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza,3,11,1
1,2,2,classic_dlx_m,1,2015-01-01,2023-03-24 11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza,3,11,1
2,3,2,five_cheese_l,1,2015-01-01,2023-03-24 11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza,3,11,1
3,4,2,ital_supr_l,1,2015-01-01,2023-03-24 11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza,3,11,1
4,5,2,mexicana_m,1,2015-01-01,2023-03-24 11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza,3,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48615,48616,21348,ckn_alfredo_m,1,2015-12-31,2023-03-24 21:23:10,16.75,16.75,M,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A...",The Chicken Alfredo Pizza,3,21,12
48616,48617,21348,four_cheese_l,1,2015-12-31,2023-03-24 21:23:10,17.95,17.95,L,Veggie,"Ricotta Cheese, Gorgonzola Piccante Cheese, Mo...",The Four Cheese Pizza,3,21,12
48617,48618,21348,napolitana_s,1,2015-12-31,2023-03-24 21:23:10,12.00,12.00,S,Classic,"Tomatoes, Anchovies, Green Olives, Red Onions,...",The Napolitana Pizza,3,21,12
48618,48619,21349,mexicana_l,1,2015-12-31,2023-03-24 22:09:54,20.25,20.25,L,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza,3,22,12


Perfect, now we can begin to take a closer look at this data.

In [9]:
df.shape

(48620, 15)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   order_details_id   48620 non-null  int64         
 1   table_number       48620 non-null  int64         
 2   pizza_id           48620 non-null  object        
 3   quantity           48620 non-null  int64         
 4   order_date         48620 non-null  datetime64[ns]
 5   order_time         48620 non-null  datetime64[ns]
 6   unit_price         48620 non-null  float64       
 7   total_price        48620 non-null  float64       
 8   pizza_size         48620 non-null  object        
 9   pizza_category     48620 non-null  object        
 10  pizza_ingredients  48620 non-null  object        
 11  pizza_name         48620 non-null  object        
 12  day_of_week        48620 non-null  int64         
 13  hour_of_day        48620 non-null  int64         
 14  month_

In [11]:
df.describe()

,order_details_id,table_number,quantity,unit_price,total_price,day_of_week,hour_of_day,month_of_sale
count,48620.000000,48620.000000,48620.000000,48620.000000,48620.000000,48620.000000,48620.000000,48620.000000
mean,24310.500000,10701.479761,1.019622,16.494132,16.821474,3.023797,15.911806,6.451131
std,14035.529381,6180.119770,0.143077,3.621789,4.437398,1.934155,3.149836,3.435420
min,1.000000,1.000000,1.000000,9.750000,9.750000,0.000000,9.000000,1.000000
25%,12155.750000,5337.000000,1.000000,12.750000,12.750000,1.000000,13.000000,3.000000
50%,24310.500000,10682.500000,1.000000,16.500000,16.500000,3.000000,16.000000,6.000000
75%,36465.250000,16100.000000,1.000000,20.250000,20.500000,5.000000,18.000000,9.000000
max,48620.000000,21350.000000,4.000000,35.950000,83.000000,6.000000,23.000000,12.000000


In [41]:
#Question: How many pizzas have sold in total?
df["quantity"].sum()

49574

In [46]:
#What is the most/least a customer will pay for a pizza?
df.groupby(["pizza_name", 'pizza_size'])['unit_price'].mean().sort_values()
df.groupby(["pizza_name", 'pizza_size'])['unit_price'].mean().sort_values(ascending=False)

pizza_name                                  pizza_size
The Greek Pizza                             XXL           35.95
                                            XL            25.50
The Brie Carre Pizza                        S             23.65
The Italian Vegetables Pizza                L             21.00
The Prosciutto and Arugula Pizza            L             20.75
                                                          ...  
The Big Meat Pizza                          S             12.00
The Vegetables + Vegetables Pizza           S             12.00
The Pepperoni, Mushroom, and Peppers Pizza  S             11.00
The Hawaiian Pizza                          S             10.50
The Pepperoni Pizza                         S              9.75
Name: unit_price, Length: 91, dtype: float64

It looks like the total sales for 2015 were $817,860.05. 48,620 orders were placed for the year and in total, the company sold 49,574 pizzas. 

The average cost of any pizza at BB's will run a customer $16.49. The most a customer will pay is $35.95 for a XXL Greek Pizza, and the least is $9.75 for a small pepperoni. The most a customer has spent at BB's is $83.00. On average, a customer order will make a sale of $16.82.

BB's top selling pizzas for the year were:

In [13]:
#It looks like these 10 pizzas brought in the most sales.
df.groupby('pizza_name')['total_price'].sum().round(2).sort_values(ascending=False).head(10)

pizza_name
The Thai Chicken Pizza          43434.25
The Barbecue Chicken Pizza      42768.00
The California Chicken Pizza    41409.50
The Classic Deluxe Pizza        38180.50
The Spicy Italian Pizza         34831.25
The Southwest Chicken Pizza     34705.75
The Italian Supreme Pizza       33476.75
The Hawaiian Pizza              32273.25
The Four Cheese Pizza           32265.70
The Sicilian Pizza              30940.50
Name: total_price, dtype: float64

**Suggestion**: Focus discounts and promotions for customers on these popular flavors.  

And which pizzas didn't sell so well?


In [14]:
df.groupby('pizza_name')['total_price'].sum().round(2).sort_values(ascending=True).head(10)

pizza_name
The Brie Carre Pizza            11588.50
The Green Garden Pizza          13955.75
The Spinach Supreme Pizza       15277.75
The Mediterranean Pizza         15360.50
The Spinach Pesto Pizza         15596.00
The Calabrese Pizza             15934.25
The Italian Vegetables Pizza    16019.25
The Soppressata Pizza           16425.75
The Chicken Pesto Pizza         16701.75
The Chicken Alfredo Pizza       16900.25
Name: total_price, dtype: float64

How many different flavors of pizza do are offered at BB's?

In [15]:
df.pizza_name.value_counts().count()

32

**Suggestion:** Could less flavor options increase sales? By eliminating options customers don't necesarrily prefer and removing the cost of making those flavors, there could be a boost in overall revenue for the year.

What do sales look like by pizza size?

In [16]:
df.groupby('pizza_size')['total_price'].sum().round(2).sort_values(ascending=False)

pizza_size
L      375318.70
M      249382.25
S      178076.50
XL      14076.00
XXL      1006.60
Name: total_price, dtype: float64

Large pizza sales seem to dominate customer orders.
Let's create a menu for the 5 top selling large flavors:

In [17]:
df.query('pizza_size == "L"').groupby(['pizza_name', 'pizza_size', 'unit_price'])['total_price'].sum().round(2).sort_values(ascending=False).head(5)

pizza_name                   pizza_size  unit_price
The Thai Chicken Pizza       L           20.75         29257.50
The Five Cheese Pizza        L           18.50         26066.50
The Four Cheese Pizza        L           17.95         23622.20
The Spicy Italian Pizza      L           20.75         23011.75
The Southwest Chicken Pizza  L           20.75         21082.00
Name: total_price, dtype: float64

How many different types of pizza are sold? What is the top selling category? What percentage of sales does each category make up of total sales?


In [18]:
#category stats
cat_sales_stats = df.groupby("pizza_category")["total_price"].agg(['min', 'max', 'mean', 'median'])
print(cat_sales_stats)

                  min    max       mean  median
pizza_category                                 
Chicken         12.75  83.00  18.115534   16.75
Classic          9.75  61.50  15.093840   14.50
Supreme         12.25  62.25  17.678271   16.50
Veggie          12.00  55.50  16.917674   16.75


In [47]:
# Four different types of pizzas at BBs: # of sales
df[["pizza_category"]].value_counts()

pizza_category
Classic           14579
Supreme           11777
Veggie            11449
Chicken           10815
dtype: int64

In [20]:
df.groupby("pizza_category")["total_price"].sum().sort_values(ascending=False)

pizza_category
Classic    220053.10
Supreme    208197.00
Chicken    195919.50
Veggie     193690.45
Name: total_price, dtype: float64

In [21]:
#sales per category
all_bb_sales = df["total_price"].sum()
all_categories = {'Classic': 'Classic', 'Supreme':'Supreme', 'Veggie': 'Veggie', 'Chicken': 'Chicken'}

classic_sales = round(df[df["pizza_category"] == 'Classic']["total_price"].sum() / all_bb_sales * 100)
supreme_sales = round(df[df["pizza_category"] == 'Supreme']["total_price"].sum() / all_bb_sales * 100)
veggie_sales = round(df[df["pizza_category"] == 'Veggie']["total_price"].sum() / all_bb_sales * 100)
chicken_sales = round(df[df["pizza_category"] == 'Chicken']["total_price"].sum() / all_bb_sales * 100)


category_sales = {}

for i in all_categories:
    if all_categories[i] == 'Supreme':
        category_sales[i] = supreme_sales
    elif all_categories[i] == 'Classic':
        category_sales[i] = classic_sales
    elif all_categories[i] == 'Veggie':
        category_sales[i] = veggie_sales
    elif all_categories[i] == 'Chicken':
        category_sales[i] = chicken_sales
    else:
        print("done")

print(category_sales)

{'Classic': 27, 'Supreme': 25, 'Veggie': 24, 'Chicken': 24}


Classic pizza options made up the most of BB's sales (14,579 sold, $220,053.10 in sales (27%)).

**Suggestion:** BB's could offer:
- Discounts on Classic pizzas
- Discounts on Classic large pizzas

to increase customer traffic and promote sales.





Which classic pizza is selling the best?


In [23]:
df.query('pizza_category == "Classic"').groupby(['pizza_name', 'pizza_size'])['total_price'].sum().round(2).head(10).sort_values(ascending=False)

pizza_name                pizza_size
The Big Meat Pizza        S             22968.0
The Classic Deluxe Pizza  M             18896.0
The Hawaiian Pizza        L             15163.5
The Greek Pizza           XL            14076.0
The Classic Deluxe Pizza  L              9696.5
                          S              9588.0
The Greek Pizza           L              5227.5
                          M              4496.0
                          S              3648.0
                          XXL            1006.6
Name: total_price, dtype: float64

What do the store's classic options look like and what ingredients do they need?

In [22]:
df.query('pizza_category == "Classic"').groupby(['pizza_name', 'pizza_ingredients'])["unit_price"].mean().sort_values(ascending=False)

pizza_name                                  pizza_ingredients                                                           
The Greek Pizza                             Kalamata Olives, Feta Cheese, Tomatoes, Garlic, Beef Chuck Roast, Red Onions    20.035633
The Italian Capocollo Pizza                 Capocollo, Red Peppers, Tomatoes, Goat Cheese, Garlic, Oregano                  17.426803
The Napolitana Pizza                        Tomatoes, Anchovies, Green Olives, Red Onions, Garlic                           16.437285
The Classic Deluxe Pizza                    Pepperoni, Mushrooms, Red Onions, Red Peppers, Bacon                            15.575952
The Pepperoni, Mushroom, and Peppers Pizza  Pepperoni, Mushrooms, Green Peppers                                             13.865127
The Hawaiian Pizza                          Sliced Ham, Pineapple, Mozzarella Cheese                                        13.317194
The Pepperoni Pizza                         Mozzarella Cheese, Pepperoni   

On average, what are the 10 top selling pizzas?

In [52]:
df.groupby(["pizza_name", "pizza_size"]).total_price.sum().sort_values(ascending=False).head(10)

pizza_name                    pizza_size
The Thai Chicken Pizza        L             29257.50
The Five Cheese Pizza         L             26066.50
The Four Cheese Pizza         L             23622.20
The Spicy Italian Pizza       L             23011.75
The Big Meat Pizza            S             22968.00
The Southwest Chicken Pizza   L             21082.00
The Barbecue Chicken Pizza    L             20584.00
The California Chicken Pizza  L             19235.25
The Classic Deluxe Pizza      M             18896.00
The Mexicana Pizza            L             17556.75
Name: total_price, dtype: float64

Store managers will need to make sure the store is stocked with enough ingredients to pzroduce all of these pizzas. What are the individual ingredients used in all pizzas they'll need to make sure they have:

In [26]:
#creating the ingredients inventory:
df["pizza_ingredients"]

0                 Sliced Ham, Pineapple, Mozzarella Cheese
1        Pepperoni, Mushrooms, Red Onions, Red Peppers,...
2        Mozzarella Cheese, Provolone Cheese, Smoked Go...
3        Calabrese Salami, Capocollo, Tomatoes, Red Oni...
4        Tomatoes, Red Peppers, Jalapeno Peppers, Red O...
                               ...                        
48615    Chicken, Red Onions, Red Peppers, Mushrooms, A...
48616    Ricotta Cheese, Gorgonzola Piccante Cheese, Mo...
48617    Tomatoes, Anchovies, Green Olives, Red Onions,...
48618    Tomatoes, Red Peppers, Jalapeno Peppers, Red O...
48619    Barbecued Chicken, Red Peppers, Green Peppers,...
Name: pizza_ingredients, Length: 48620, dtype: object

In [27]:
inventory = []
list_of_ingredients = df["pizza_ingredients"]
# print(list_of_ingredients)
for n in list_of_ingredients:
#  print(n.split(","))
 inventory.append(n.split(", "))

# print([olist[5]])

def remove_duplicates(pizzas):
    ingredients = set()
    for pizza in inventory:
        for ingredient in pizza:
            ingredients.add(ingredient)
    return ingredients

unique_ingredients = remove_duplicates(inventory)
print(unique_ingredients)

{'Alfredo Sauce', 'Bacon', 'Peperoncini verdi', 'Brie Carre Cheese', 'Plum Tomatoes', 'Anchovies', 'Smoked Gouda Cheese', 'Arugula', 'Luganega Sausage', 'Spinach', 'Provolone Cheese', 'Beef Chuck Roast', 'Jalapeno Peppers', 'Corn', 'Green Olives', 'Oregano', 'Ricotta Cheese', 'Pesto Sauce', 'Artichokes', 'Soppressata Salami', 'Friggitello Peppers', 'Pancetta', 'Chorizo Sausage', '慛duja Salami', 'Calabrese Salami', 'Coarse Sicilian Salami', 'Caramelized Onions', 'Red Peppers', 'Chicken', 'Asiago Cheese', 'Italian Sausage', 'Parmigiano Reggiano Cheese', 'Mozzarella Cheese', 'Gouda Cheese', 'Zucchini', 'Barbecued Chicken', 'Red Onions', 'Pears', 'Fontina Cheese', 'Genoa Salami', 'Eggplant', 'Prosciutto', 'Thyme', 'Pineapple', 'Mushrooms', 'Blue Cheese', 'Kalamata Olives', 'Gorgonzola Piccante Cheese', 'Barbecue Sauce', 'Garlic', 'Goat Cheese', 'Onions', 'Chipotle Sauce', 'Sliced Ham', 'Tomatoes', 'Prosciutto di San Daniele', 'Cilantro', 'Green Peppers', 'Pepperoni', 'Feta Cheese', 'Capoco

Store managers will also need an estimate number of pizza boxes to have in the store for orders.


In [28]:
all_box_sales = df["total_price"].sum()
# print(f'The total amount of box sales is ${all_box_sales}')
box_sizes = {"S":"S", "M":"M", 'L':"L", 'XL':"XL", 'XXL': 'XXL'}

# num_of_boxes_sales =  df.groupby("pizza_size")["total_price"].value_counts().sum()
# print(num_of_boxes_sales)


# num of box sizes
for i in box_sizes:
        pizza_size = box_sizes[i]
        num_of_boxes = df[df["pizza_size"] == pizza_size]["total_price"].value_counts().sum() 
        print(f' # of {pizza_size} boxes: {num_of_boxes}')


# #sales per size of box
# for i in box_sizes:
#         pizza_size = box_sizes[i]
#         sales_of_boxes = df[df["pizza_size"] == pizza_size]["total_price"].sum()
#         print(f'{pizza_size} box sales: ${sales_of_boxes.round(2)}')

#pct of sales by pizza_size
# for i in box_sizes:
#         pizza_size = box_sizes[i]
#         pct_of_sizes = (sales_of_boxes / all_box_sales) * 100
#         print(f'{pizza_size} box sales make up {pct_of_sizes.round(2)}% of sales')


#medium boxes
# num_of_m_boxes = df[df["pizza_size"] == 'M']["total_price"].value_counts().sum() 
# print(num_of_m_boxes)
# sales_of_m_boxes = df[df["pizza_size"] == 'M']["total_price"].sum()
# print(sales_of_m_boxes)
# pct_of_msizes = sales_of_m_boxes / all_box_sales * 100
# print(pct_of_msizes)

 # of S boxes: 14137
 # of M boxes: 15385
 # of L boxes: 18526
 # of XL boxes: 544
 # of XXL boxes: 28


With a total of 49,574 pizzas sold, it would be safe for managers to order at least 14,140 S boxes, 15,390 M boxes, 18,530 large boxes, 540 XL boxes, and 30 XL boxes.

Next, we'll start gathering the data we'll need for the interactive dashboard. Let's first handle the stakeholders requests: When is the store getting the most customer orders and making the most sales?  


In [49]:
#What day of the week do we get the most orders
df["day_of_week"].value_counts().sort_values(ascending=False)
# #what time of the day?
df["hour_of_day"].value_counts().sort_values(ascending=False)
# what month?
# df["month_of_sale"].value_counts().sort_values(ascending=False)


4    8106
5    7355
3    7323
2    6797
1    6753
0    6369
6    5917
Name: day_of_week, dtype: int64

In [30]:
df.groupby("month_of_sale")["total_price"].sum().sort_values(ascending = False)

month_of_sale
7     72557.90
5     71402.75
3     70397.10
11    70395.35
1     69793.30
4     68736.80
8     68278.25
6     68230.20
2     65159.60
12    64701.15
9     64180.05
10    64027.60
Name: total_price, dtype: float64

It looks like peak store sales and orders were from May to September with a decline until picking back up in November. 

**Suggestion**: Marketing could use this data to plan seasonal campaigns to engage customers and increase sales.

In [31]:
df.groupby("hour_of_day")["total_price"].sum().sort_values(ascending = False)

hour_of_day
12    111877.90
13    106065.70
18     89296.85
17     86237.45
19     72628.90
16     70055.40
14     59201.40
20     58215.40
15     52992.30
11     44935.80
21     42029.80
22     22815.15
23      1121.35
10       303.65
9         83.00
Name: total_price, dtype: float64

In [32]:
df.groupby("day_of_week")["total_price"].sum().sort_values(ascending = False)

day_of_week
4    136073.90
3    123528.50
5    123182.40
2    114408.40
1    114133.80
0    107329.55
6     99203.50
Name: total_price, dtype: float64

The most sales and customer orders are made on Thursdays. Peak sale hours are from 12PM to 1PM with a decline until picking back up from 5PM to 6PM. It look's like the lunch rush and the return from work are two of the most popular times for sales. 

**Suggestion**: The marketing and sales departments could work on offering a lunch menu discount or dinner menu combo on Thursday's to boost sales.

In [33]:
# name stats
name_sales_stats = df.groupby("pizza_name")["total_price"].agg(['min', 'max', 'mean', 'median'])
print(name_sales_stats)

                                              min    max       mean  median
pizza_name                                                                 
The Barbecue Chicken Pizza                  12.75  62.25  18.030354   16.75
The Big Meat Pizza                          12.00  48.00  12.682496   12.00
The Brie Carre Pizza                        23.65  47.30  24.142708   23.65
The Calabrese Pizza                         12.25  40.50  17.189051   16.25
The California Chicken Pizza                12.75  83.00  17.988488   16.75
The Chicken Alfredo Pizza                   12.75  41.50  17.245153   16.75
The Chicken Pesto Pizza                     12.75  41.50  17.379553   16.75
The Classic Deluxe Pizza                    12.00  48.00  15.803187   16.00
The Five Cheese Pizza                       18.50  55.50  19.180648   18.50
The Four Cheese Pizza                       14.75  35.90  17.440919   17.95
The Greek Pizza                             12.00  51.00  20.237624   20.50
The Green Ga

In [34]:
#details stats
details_sales_stats =  df.groupby("pizza_id")["total_price"].agg(['min', 'max', 'mean', 'median'])
print(details_sales_stats.head(10))

                min    max       mean  median
pizza_id                                     
bbq_ckn_l     20.75  62.25  21.286453   20.75
bbq_ckn_m     16.75  50.25  17.292657   16.75
bbq_ckn_s     12.75  25.50  12.883090   12.75
big_meat_s    12.00  48.00  12.682496   12.00
brie_carre_s  23.65  47.30  24.142708   23.65
calabrese_l   20.25  40.50  20.397810   20.25
calabrese_m   16.25  32.50  16.484657   16.25
calabrese_s   12.25  12.25  12.250000   12.25
cali_ckn_l    20.75  83.00  21.491899   20.75
cali_ckn_m    16.75  33.50  17.299781   16.75


In [35]:
df.to_csv('pizzasales.csv')

The stakeholders would like for a dashboard to be created making sure to include:
- Customer orders and sales
- Top selling items 
- Monthly and daily sales

 This interactive dashboard in [Tableau](https://public.tableau.com/views/pizza_sales_16794579121300/pizza?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link) details just that and more.